In [ ]:
import tensorflow as tf
import os
import zipfile
from os import path, getcwd, chdir
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator

path = f"{getcwd()}/../tmp2/happy-or-sad.zip"

zip_ref = zipfile.ZipFile(path, 'r')
zip_ref.extractall("/tmp/h-or-s")
zip_ref.close()

In [ ]:
# GRADED FUNCTION: train_happy_sad_model
def train_happy_sad_model():

    DESIRED_ACCURACY = 0.999

    class myCallback(tf.keras.callbacks.Callback):
         def on_epoch_end(self, epoch, logs={}):
                if(logs.get("acc") > DESIRED_ACCURACY):
                    print('\nReached 99.9% accuracy so cancelling training!')
                    self.model.stop_training = True

    callbacks = myCallback()
    
    model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(150, 150, 3)),
        tf.keras.layers.MaxPooling2D(2, 2),
        # The second convolution
        tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2,2),
        # The third convolution
        tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])

    model.compile(loss="binary_crossentropy", optimizer=RMSprop(lr=0.001), metrics=["accuracy"])

    train_datagen = ImageDataGenerator(rescale = 1/255)

    train_generator = train_datagen.flow_from_directory(
        "/tmp/h-or-s",
        target_size=(150,150),
        batch_size=128,
        class_mode="binary")
    
    history = model.fit_generator(
        train_generator,
        steps_per_epoch=1,
        epochs=30,
        verbose=1,
        callbacks=[callbacks])
    
    # model fitting
    return history.history['acc'][-1]

In [ ]:
# The Expected output: "Reached 99.9% accuracy so cancelling training!""
train_happy_sad_model()